In [1]:
# define the neural netowrk that has some learnable parameters (or weights)
# iterate over a dataset of inputs
# process input through the network
# compute the loss
# propagate gradients back into the network's parameters
# update the weights of the network, typically using a simple update rule: weight = weight - learning_date * gradient

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
from functools import reduce
import operator

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(5 * 5 * 16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def num_flatten(self, x):
        size = x.size()[1:] # ignore batch size
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # Flatten
        x = x.view(-1, reduce(operator.mul, x.size()[1:], 1))
        x = F.relu(self.fc1(x))
        # relu -> tanh
        x = torch.tanh(self.fc2(x))
        x = self.fc3(x) # what is default activation function
        return x

In [4]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [5]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's .weight

10
torch.Size([6, 1, 5, 5])


In [6]:
# input data

In [25]:
input = torch.randn(1, 1, 32, 32) # batch, channel, width, height
out = net(input)
print(out)

tensor([[-0.0476,  0.0920, -0.0194, -0.0357, -0.0387,  0.0885, -0.0651, -0.0472,
         -0.0128, -0.0425]], grad_fn=<AddmmBackward>)


In [20]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [14]:
# Losses
out = net(input)

In [26]:
target = torch.randn((1, 10))
# target = target.unsqueeze(0) or
# target = target.view(1, -1) on target = torch.randn(10)
criterion = nn.MSELoss()
loss = criterion(out, target)
print(loss)

tensor(1.6879, grad_fn=<MseLossBackward>)


In [27]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [28]:
# backpropagation

In [29]:
net.zero_grad() # zeroes the gradient buffers of all parameters
print('conv1.bias.grad before backword')
print(net.conv1.bias.grad)

loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backword
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0115, -0.0178,  0.0086, -0.0243,  0.0038, -0.0014])


In [ ]:
# update the weights
# weight = weight - (learning rate) * gradient

In [30]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [32]:
import torch.optim as optim

# create optimizer
optimizer = optim.SGD(net.parameters(), lr = 0.01)

# in tranining loop
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()